<a href="https://colab.research.google.com/github/nikbullk/awesome-fonts/blob/master/MOMO_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Практическое задание 3: Одномерный анализ данных для улучшения
качества данных

Душкина А.А.

....

In [ ]:
# Скачиваем необоходимые библиотеки
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,auc,classification_report
from sklearn.metrics import RocCurveDisplay, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, roc_curve

В используемом датасете содержится информация о потреблении энергии и воды в Нью-Йорке.

Краткое описание используемых переменных:

Property Id - Номер, присвоенный каждой заявке на бенчмаркинг. Это значение уникально для каждой записи, представленной в поле BBL. Если Департамент финансов не получил заявку на бенчмаркинг, то в соответствующей колонке не будет присвоенного номера записи

BBL - 10 digits - 10-значный идентификатор недвижимости, состоящий из округа, блока и участка

Postal Code - Почтовый ZIP-код недвижимости, указанный самостоятельно

Self-Reported Gross Floor Area (ft²) - Варианты типов недвижимости

Year Built - Год постройки недвижимости

Number of Buildings - Количество зданий указывает общее число зданий, расположенных на налоговом участке

Occupancy - Процент общей площади (GFA) вашей недвижимости, которая занята и функционирует

ENERGY STAR Score - Рейтинг от 1 до 100 для определённых типов зданий

Source EUI (kBtu/ft²) - Интенсивность потребления энергии, выраженная в кБту на квадратный фут общей площади за отчётный год

Weather Normalized Source EUI (kBtu/ft²) - Интенсивность потребления энергии, выраженная в кБту на квадратный фут общей площади за отчётный год, с учетом нормализации по погодным условиям

Site EUI (kBtu/ft²) - Интенсивность потребления энергии, выраженная в кБту на квадратный фут общей площади за отчётный год

Weather Normalized Site EUI (kBtu/ft²) - Интенсивность потребления энергии, выраженная в кБту на квадратный фут общей площади за отчётный год, с учетом нормализации по погодным условиям

Weather Normalized Site Electricity Intensity (kWh/ft²) - Нормализованное по погодным условиям потребление энергии на объекте, делённое на размер недвижимости или на поток через водо- или водоотводное очистное сооружение.

Weather Normalized Site Natural Gas Intensity (therms/ft²) - Нормализованное по погодным условиям потребление энергии на объекте, делённое на размер недвижимости или на поток через водо- или водоотводное очистное сооружение.

Детальнее про прочие переменные см., пожауйлста, в пдф файле

In [ ]:
# Скачиваем датасет
df = pd.read_excel('/content/nyc_benchmarking_disclosure_2017_consumption_data.xlsx', sheet_name=1)

In [ ]:
# Просмотр переменных
df

,Order,Property Id,Property Name,Parent Property Id,Parent Property Name,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Address 2 (self-reported),...,Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Water Use (All Water Sources) (kgal),Water Use Intensity (All Water Sources) (gal/ft²),Water Required?,Generation Date,DOF Benchmarking Submission Status
0,1,4593574,The Argonaut Building,NaN,NaN,1.010288e+09,1010287502,1024898,224 West 57th St,NaN,...,NaN,NaT,732.4,76.3,656.1,3635.5,21.46,Not found,2018-02-14,Not found
1,3,2967701,Cathedral Preparatory Seminary,NaN,NaN,4.018720e+09,4-01872-0007,4046340,56-25 92nd Street,NaN,...,NaN,NaT,164.5,109.9,54.6,102.9,1.09,Not found,2018-02-14,Not found
2,4,4898531,The Nomad Hotel,NaN,NaN,1.008290e+09,1-00829-0050,1080710,1170 Broadway,NaN,...,NaN,NaT,1150.2,438.0,712.3,10762.6,86.10,Not found,2018-02-14,Not found
3,5,2917939,10 West 27 Street Corp,NaN,NaN,1.008280e+09,1-00828-0053,1015657,1155 Broadway,NaN,...,NaN,NaT,273.4,98.2,175.3,790.1,15.80,Not found,2018-02-14,Not found
4,6,3878205,Westbury Realty,NaN,NaN,1.000650e+09,1-00065-0024,1001105,24 John Street,NaN,...,NaN,NaT,90.7,0.0,90.7,143.0,2.86,Not found,2018-02-14,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34350,34682,6682473,Mitchell Hill- 2802 Clarendon Road,NaN,NaN,3.051940e+09,3-05194-0001,3119966,2802 Clarendon Road,NaN,...,NaN,NaT,187.1,160.2,26.8,295851.5,9019.86,Not found,2019-03-07,Not found
34351,34683,6682474,Mitchell Hill- 1116 Carroll Street,NaN,NaN,3.012880e+09,3-01288-0038,3033818,1116 Carroll Street,NaN,...,NaN,NaT,181.2,157.7,23.5,283262.5,8756.18,Not found,2019-03-07,Not found
34352,34684,6682477,Mitchell Hill- 3033 Brighton 13 Street,NaN,NaN,3.087160e+09,3-08716-0085,3245653,3033 Brighton 13 Street,NaN,...,NaN,NaT,230.4,177.0,53.3,451795.0,13366.71,Not found,2019-03-07,Not found
34353,34685,2819133,The Ritz Tower - 465 Park Avenue,NaN,NaN,1.013128e+09,1-01312-7502,1036922,465 Park Avenue,NaN,...,NaN,NaT,614.8,23.7,591.1,7118.2,27.04,Not found,2019-03-08,Not found


In [ ]:
# В таблице 34355 строки и 60 столбцов
print(df.shape)

(34355, 60)


In [ ]:
# Названия столбцов
print(df.columns)

Index(['Order', 'Property Id', 'Property Name', 'Parent Property Id',
       'Parent Property Name', 'BBL - 10 digits',
       'NYC Borough, Block and Lot (BBL) self-reported',
       'NYC Building Identification Number (BIN)', 'Address 1 (self-reported)',
       'Address 2 (self-reported)', 'Postal Code', 'Street Number',
       'Street Name', 'Borough', 'DOF Gross Floor Area (ft²)',
       'Self-Reported Gross Floor Area (ft²)',
       'Primary Property Type - Self Selected',
       'List of All Property Use Types at Property',
       'Largest Property Use Type',
       'Largest Property Use Type - Gross Floor Area (ft²)',
       '2nd Largest Property Use Type',
       '2nd Largest Property Use - Gross Floor Area (ft²)',
       '3rd Largest Property Use Type',
       '3rd Largest Property Use Type - Gross Floor Area (ft²)', 'Year Built',
       'Number of Buildings', 'Occupancy', 'Metered Areas (Energy)',
       'Metered Areas  (Water)', 'ENERGY STAR Score', 'Source EUI (kBtu/ft²)',


In [ ]:
# Общ инфа
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34355 entries, 0 to 34354
Data columns (total 60 columns):
 #   Column                                                      Non-Null Count  Dtype         
---  ------                                                      --------------  -----         
 0   Order                                                       34355 non-null  int64         
 1   Property Id                                                 34355 non-null  int64         
 2   Property Name                                               34355 non-null  object        
 3   Parent Property Id                                          1723 non-null   object        
 4   Parent Property Name                                        1723 non-null   object        
 5   BBL - 10 digits                                             33625 non-null  float64       
 6   NYC Borough, Block and Lot (BBL) self-reported              33626 non-null  object        
 7   NYC Building Identific

In [ ]:
# Статистика по нечисловым признакам
df.describe(include=['object', 'bool'])

,Property Name,Parent Property Id,Parent Property Name,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Address 2 (self-reported),Postal Code,Street Number,Street Name,...,DOF Gross Floor Area (ft²),Primary Property Type - Self Selected,List of All Property Use Types at Property,Largest Property Use Type,2nd Largest Property Use Type,3rd Largest Property Use Type,Metered Areas (Energy),Metered Areas (Water),Water Required?,DOF Benchmarking Submission Status
count,34355,1723,1723,33626,33123,34355,427,34355,34355,34355,...,34355,34355,34342,34323,7226,2450,31710,14546,34355,34355
unique,22501,176,176,21495,21078,22062,242,330,1,1,...,1,69,1035,66,60,51,27,5,1,1
top,East Building,3612678,Columbia University (morningside),1019730001,0,410 West 118th Street,B-230,10027,Not found,Not found,...,Not found,Multifamily Housing,Multifamily Housing,Multifamily Housing,Parking,Retail Store,Whole Building,Whole Building,Not found,Not found
freq,20,395,395,182,54,80,52,1037,34355,34355,...,34355,23399,18836,23386,1971,475,31397,14493,34355,34355


In [ ]:
# df['Property Id'].value_counts()

In [ ]:
# df['Property Id'].value_counts(normalize=True)

In [ ]:
# Iterate through the columns
# for col in list(data.columns):
#     # Select columns that should be numeric
#     if ('ft²' in col or 'kBtu' in col or 'Metric Tons CO2e' in col or 'kWh' in
#         col or 'therms' in col or 'gal' in col or 'Score' in col):
#         # Convert the data type to float
#         data[col] = data[col].astype(float)

In [ ]:
# data

,Order,Property Id,Property Name,Parent Property Id,Parent Property Name,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Address 2 (self-reported),...,Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Water Use (All Water Sources) (kgal),Water Use Intensity (All Water Sources) (gal/ft²),Water Required?,Generation Date,DOF Benchmarking Submission Status
0,1,4593574,The Argonaut Building,NaN,NaN,1.010288e+09,1010287502,1024898,224 West 57th St,NaN,...,NaN,NaT,732.4,76.3,656.1,3635.5,21.46,NaN,2018-02-14,NaN
1,3,2967701,Cathedral Preparatory Seminary,NaN,NaN,4.018720e+09,4-01872-0007,4046340,56-25 92nd Street,NaN,...,NaN,NaT,164.5,109.9,54.6,102.9,1.09,NaN,2018-02-14,NaN
2,4,4898531,The Nomad Hotel,NaN,NaN,1.008290e+09,1-00829-0050,1080710,1170 Broadway,NaN,...,NaN,NaT,1150.2,438.0,712.3,10762.6,86.10,NaN,2018-02-14,NaN
3,5,2917939,10 West 27 Street Corp,NaN,NaN,1.008280e+09,1-00828-0053,1015657,1155 Broadway,NaN,...,NaN,NaT,273.4,98.2,175.3,790.1,15.80,NaN,2018-02-14,NaN
4,6,3878205,Westbury Realty,NaN,NaN,1.000650e+09,1-00065-0024,1001105,24 John Street,NaN,...,NaN,NaT,90.7,0.0,90.7,143.0,2.86,NaN,2018-02-14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34350,34682,6682473,Mitchell Hill- 2802 Clarendon Road,NaN,NaN,3.051940e+09,3-05194-0001,3119966,2802 Clarendon Road,NaN,...,NaN,NaT,187.1,160.2,26.8,295851.5,9019.86,NaN,2019-03-07,NaN
34351,34683,6682474,Mitchell Hill- 1116 Carroll Street,NaN,NaN,3.012880e+09,3-01288-0038,3033818,1116 Carroll Street,NaN,...,NaN,NaT,181.2,157.7,23.5,283262.5,8756.18,NaN,2019-03-07,NaN
34352,34684,6682477,Mitchell Hill- 3033 Brighton 13 Street,NaN,NaN,3.087160e+09,3-08716-0085,3245653,3033 Brighton 13 Street,NaN,...,NaN,NaT,230.4,177.0,53.3,451795.0,13366.71,NaN,2019-03-07,NaN
34353,34685,2819133,The Ritz Tower - 465 Park Avenue,NaN,NaN,1.013128e+09,1-01312-7502,1036922,465 Park Avenue,NaN,...,NaN,NaT,614.8,23.7,591.1,7118.2,27.04,NaN,2019-03-08,NaN


In [ ]:
# Заменяем Not Available
df = df.replace({'Not found': np.nan})

<ipython-input-23-dae3f2ed4952>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'Not found': np.nan})


In [ ]:
# df = df[['Property Id', 'BBL - 10 digits', 'Postal Code', 'Self-Reported Gross Floor Area (ft²)', 'Year Built', 'Number of Buildings', 'Occupancy', 'ENERGY STAR Score', 'Source EUI (kBtu/ft²)', 'Weather Normalized Source EUI (kBtu/ft²)', 'Site EUI (kBtu/ft²)', 'Weather Normalized Site EUI (kBtu/ft²)', 'Weather Normalized Site Electricity Intensity (kWh/ft²)', 'Weather Normalized Site Natural Gas Intensity (therms/ft²)']]

In [ ]:
# Удаляем некоторые переменные
df = df.drop(columns=['Order', 'Property Name', 'Parent Property Id', 'Parent Property Name', 'NYC Borough, Block and Lot (BBL) self-reported', 'NYC Building Identification Number (BIN)', 'Address 1 (self-reported)', 'Address 2 (self-reported)', 'Street Number', 'Street Name', 'Borough', 'DOF Gross Floor Area (ft²)', 'Primary Property Type - Self Selected', 'List of All Property Use Types at Property', 'Largest Property Use Type', 'Largest Property Use Type - Gross Floor Area (ft²)', '2nd Largest Property Use Type', '2nd Largest Property Use - Gross Floor Area (ft²)', '3rd Largest Property Use Type', '3rd Largest Property Use Type - Gross Floor Area (ft²)', 'Metered Areas (Energy)', 'Metered Areas  (Water)', 'Water Required?', 'Generation Date', 'DOF Benchmarking Submission Status', 'Annual Maximum Demand (MM/YYYY)', 'Source EUI (kBtu/ft²)'])

In [ ]:
# df.drop(columns=['Generation Date'], inplace=True)
# df = df.drop(columns=['Generation Date', 'DOF Benchmarking Submission Status', 'Water Required?', 'DOF Gross Floor Area (ft²)', 'Street Name', 'Street Number', 'Order', 'Property Name', 'Address 1 (self-reported)', 'Address 2 (self-reported)', 'List of All Property Use Types at Property', 'Largest Property Use Type', '2nd Largest Property Use Type', '3rd Largest Property Use Type - Gross Floor Area (ft²)'])
# df

In [ ]:
# Преобразования категориальных признаков в бинарные
df = pd.get_dummies(df, drop_first=True)
# df

In [ ]:
# Осн стат характеристики
describe = df.describe()
skewness = df.skew()  # Асимметрия
kurtosis = df.kurt()  # Эксцесс

numeric_df = df.select_dtypes(include=['int64', 'float64'])
range_values = numeric_df.max() - numeric_df.min()
median = df.median()

# Печатаем результаты
print("Описательная статистика:")
print(describe)

print("\nАсимметрия:")
print(skewness)

print("\nЭксцесс:")
print(kurtosis)

print("\nРазмах:")
print(range_values)

Описательная статистика:
        Property Id  BBL - 10 digits  Self-Reported Gross Floor Area (ft²)  \
count  3.435500e+04     3.362500e+04                          3.435500e+04   
mean   4.710374e+06     2.249470e+09                          1.149392e+05   
std    1.676669e+06     1.193104e+09                          2.164591e+05   
min    7.365000e+03     0.000000e+00                          0.000000e+00   
25%    2.825644e+06     1.014980e+09                          4.227300e+04   
50%    4.897531e+06     2.032480e+09                          6.582000e+04   
75%    6.297280e+06     3.058740e+09                          1.136150e+05   
max    6.716654e+06     7.000500e+09                          1.507766e+07   

         Year Built  Number of Buildings     Occupancy  ENERGY STAR Score  \
count  34355.000000         34355.000000  34355.000000       25454.000000   
mean    1946.106855             1.196565     98.393101          53.927163   
std       32.725881             2.959191 

In [ ]:
# describe

,Property Id,BBL - 10 digits,Self-Reported Gross Floor Area (ft²),Year Built,Number of Buildings,Occupancy,ENERGY STAR Score,Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),Site EUI (kBtu/ft²),...,Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Weather Normalized Site Electricity (kWh),Annual Maximum Demand (kW),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Water Use (All Water Sources) (kgal),Water Use Intensity (All Water Sources) (gal/ft²)
count,3.435500e+04,3.362500e+04,3.435500e+04,34355.000000,34355.000000,34355.000000,25454.000000,3.188800e+04,2.856000e+04,3.188800e+04,...,2.804700e+04,3.157600e+04,3.157600e+04,3.101300e+04,2.383000e+03,3.295700e+04,3.311600e+04,3.308800e+04,1.828000e+04,1.827100e+04
mean,4.710374e+06,2.249470e+09,1.149392e+05,1946.106855,1.196565,98.393101,53.927163,1.265785e+03,1.393057e+03,9.922633e+02,...,3.576895e+05,3.833599e+06,1.123563e+06,1.126104e+06,2.635069e+03,1.893548e+04,1.665273e+03,1.719766e+04,9.964984e+03,5.869470e+02
std,1.676669e+06,1.193104e+09,2.164591e+05,32.725881,2.959191,8.917131,32.573629,1.626921e+05,1.719455e+05,1.349948e+05,...,3.329299e+07,1.249719e+07,3.662716e+06,3.674892e+06,5.617472e+04,2.954070e+06,1.621715e+05,2.943758e+06,9.190300e+04,5.251308e+04
min,7.365000e+03,0.000000e+00,0.000000e+00,1051.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-6.343300e+03,-1.859100e+03,-1.859100e+03,0.000000e+00,0.000000e+00,0.000000e+00,-1.760050e+04,0.000000e+00,0.000000e+00
25%,2.825644e+06,1.014980e+09,4.227300e+04,1925.000000,1.000000,100.000000,25.000000,9.350000e+01,9.750000e+01,5.800000e+01,...,8.581150e+03,6.888429e+05,2.018883e+05,2.022287e+05,4.560000e+01,1.841000e+02,6.040000e+01,5.380000e+01,1.954200e+03,2.847000e+01
50%,4.897531e+06,2.032480e+09,6.582000e+04,1935.000000,1.000000,100.000000,56.000000,1.224000e+02,1.276000e+02,7.990000e+01,...,3.226560e+04,1.249026e+06,3.660686e+05,3.673331e+05,1.760000e+02,3.283000e+02,1.888000e+02,1.024000e+02,3.794200e+03,5.003000e+01
75%,6.297280e+06,3.058740e+09,1.136150e+05,1965.000000,1.000000,100.000000,83.000000,1.628000e+02,1.689000e+02,1.034000e+02,...,5.846190e+04,2.828563e+06,8.290043e+05,8.342771e+05,3.080000e+02,5.962000e+02,3.313000e+02,2.387250e+02,6.631100e+03,8.339500e+01
max,6.716654e+06,7.000500e+09,1.507766e+07,2021.000000,161.000000,100.000000,100.000000,2.894294e+07,2.894295e+07,2.405806e+07,...,3.936197e+09,5.742834e+08,1.683128e+08,1.676955e+08,2.553601e+06,5.354297e+08,2.083388e+07,5.354297e+08,5.446590e+06,6.913227e+06


Occupancy: средняя % от общей площади объекта недвижимости, которая занята и находится в эксплуатации, составляет 98.39; стандартное отклонение занятой недвижимости - 8.92; размах равен 1.000000e+02 (относительно не большой)

ENERGY STAR Score: средняя процентильное ранжирование от 1 до 100 равно 53.93, стандартное отклонение равно 32.57, а размах - 9.900000e+01

Weather Normalized Site Electricity Intensity: средняя трат на энергию - 3.58,
стандартное отклонение - 3.67, размах - 1.361577e+05

Weather Normalized Site Gas Intensity:средняя трат на газ - 3.58, стандартное отклонение - 3.33, размах - 9.258400e+03

Weather Normalized Source EUI (kBtu/ft²): средняя интенсивности энергопотребления с учетом погодных условий - 1.39, стандартное отклонение - 1.72, размах - 1.361577e+05

Site EUI (kBtu/ft²): средняя интенсивности энергопотребления равна 1.27, стандартное отклонение - 1.63, размах - 2.894294e+07

Electricity Use - Grid Purchase (kWh): средняя потребления энергии - 1.12, стандартное отклонение - 3.66, размах - 1.683147e+08

Weather Normalized Site Electricity (kWh): среднее количество энергии, которое было бы израсходовано в течение средних погодных условий за 30 лет - 1.13, стандартное отклонение - 3,67, размах - 1.676974e+08

Annual Maximum Demand - средняя максимального годового потребления - 2.64, стандартное отклонение - 5.62, размах - 2.553601e+06

Total GHG Emissions (Metric Tons CO2e): средняя общего количество прямых и косвенных выбросов парниковых газов объектом - 1.89, стандартное отклонение - 2.95, размах - 5.354297e+08

Water Use (All Water Sources) (kgal): средняя суммы показаний всех счетчиков воды - 9.96, стандартное отклонение - 9.19, размах - 5.446590e+06

In [ ]:
X = df.drop(columns = ['Property Id'])
y = df['Property Id']

In [ ]:
# Делим данные на тест и трейн
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# from sklearn.impute import SimpleImputer

# X = df.drop(columns = ['Property Id'])
# y = df['Property Id']
# Сохраняем имена столбцов
# columns = X.columns

# Инициализируем SimpleImputer с выбранной стратегией
# imputer = SimpleImputer(strategy='mean')  # Можно использовать 'median' или 'most_frequent'

# Применяем SimpleImputer к данным X (обучающие данные)
# X_train = imputer.fit_transform(X)  # fit_transform для обучающего набора данных

# Применяем SimpleImputer к X_test (тестовые данные)
# (Предположим, что X_test уже есть, и он имеет те же признаки)
# X_test = imputer.transform(X)  # transform для тестового набора

# Восстанавливаем DataFrame и добавляем имена столбцов
# X_train = pd.DataFrame(X_train, columns=columns)
# X_test = pd.DataFrame(X_test, columns=columns)

# Теперь проверяем на пропуски
# rint("Пропуски в X_train:")
# print(X_train.isnull().sum())

# print("Пропуски в X_test:")
# print(X_test.isnull().sum())

Пропуски в X_train:
BBL - 10 digits                         0
Self-Reported Gross Floor Area (ft²)    0
Year Built                              0
Number of Buildings                     0
Occupancy                               0
                                       ..
Postal Code_11236-2120                  0
Postal Code_11378-2006                  0
Postal Code_11385-6201                  0
Postal Code_Bronx                       0
Postal Code_`10011                      0
Length: 341, dtype: int64
Пропуски в X_test:
BBL - 10 digits                         0
Self-Reported Gross Floor Area (ft²)    0
Year Built                              0
Number of Buildings                     0
Occupancy                               0
                                       ..
Postal Code_11236-2120                  0
Postal Code_11378-2006                  0
Postal Code_11385-6201                  0
Postal Code_Bronx                       0
Postal Code_`10011                      0
Length: 341

In [ ]:
# Заполняем все NaN значения в X_train и X_test значением 0
X_train = pd.DataFrame(X_train).fillna(0)
X_test = pd.DataFrame(X_test).fillna(0)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# print(df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34355 entries, 0 to 34354
Columns: 342 entries, Property Id to Postal Code_`10011
dtypes: bool(329), float64(8), int64(5)
memory usage: 14.2 MB
None


In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("Mean Squared Error:", mse)

Mean Squared Error: 2376638562496.4883


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2:.4f}")

# RMSE (корень из MSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.4f}")

R-squared: 0.1511
RMSE: 1541635.0290


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Используем более быстрый оптимизатор 'saga' или 'lbfgs'
model = LogisticRegression(solver='saga', max_iter=1000)  # Можно увеличить max_iter для сложных задач
model.fit(X_train, y_train)

# Предсказания
y_pred_class = model.predict(X_test)  # Классы
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Вероятности положительного класса

# Метрики
f1 = f1_score(y_test, y_pred_class)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# ROC-кривая
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label=f'ROC-AUC = {roc_auc:.2f}')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()


In [ ]:
# чтобы ускорить модель можно выбрать 20 л3чших признаков
# from sklearn.feature_selection import SelectKBest, f_classif

# selector = SelectKBest(f_classif, k=20)  # Выбираем 20 лучших признаков
# X_train_reduced = selector.fit_transform(X_train, y_train)
# X_test_reduced = selector.transform(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 15  16  18  31  32  33  34  36  37  38  39  40  41  42  45  46  47  49
  50  51  52  53  54  56  57  59  61  62  63  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 195 196 197 198 199 200
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 232 233 234 235 237 238
 239 240 241 242 243 244 245 246 247 248 250 251 252 253 254 255 256 257
 258 2

In [ ]:
# Преобразовываем данные в float32. Это снижает объем памяти и ускоряет вычисления
# import numpy as np

# X_train = X_train.astype(np.float32)
# X_test = X_test.astype(np.float32)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np

# Преобразуем данные в float32 для ускорения
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Сокращаем число признаков
selector = SelectKBest(f_classif, k=20)  # Выбираем 20 лучших признаков
X_train_reduced = selector.fit_transform(X_train, y_train)
X_test_reduced = selector.transform(X_test)

# Логистическая регрессия с ускоренными настройками
model = LogisticRegression(solver='saga', max_iter=500, n_jobs=-1)
model.fit(X_train_reduced, y_train)

# Предсказания
y_pred_class = model.predict(X_test_reduced)  # Классы
y_pred_proba = model.predict_proba(X_test_reduced)[:, 1]  # Вероятности положительного класса

# Метрики
f1 = f1_score(y_test, y_pred_class)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# ROC-кривая
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label=f'ROC-AUC = {roc_auc:.2f}')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  2  15  16  31  32  33  34  36  37  38  39  40  41  42  45  46  47  49
  50  51  52  53  54  56  57  59  61  62  63  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 195 196 197 198 199 200
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 232 233 234 235 237 238
 239 240 241 242 243 244 245 246 247 248 250 251 252 253 254 255 256 257
 258 2